In [1]:
import numpy as np

## Linear Case

$$
\tau(n) = \frac{1}{n(n-1)}\sum_{u+v=n}(au+bv)\tau(u)\tau(v)
$$
Due to symmetry, we can rewrite the sum as
$$
\tau(n) = \frac{1}{n(n-1)}\sum_{u+v=n}au\tau(u)\tau(v).
$$

In [52]:
def deg_1_array(a_min,a_max,seq_length):
    a_len = a_max+1 - a_min
    
    a_range = np.arange(a_min, a_max+1)
    u_range = np.arange(seq_length)
    
    au_array = np.multiply.outer(a_range,u_range)
    
    return au_array

def deg_1_sqnc(a_min,a_max,seq_length):
    a_len = a_max+1 - a_min
    
    coeff_array = deg_1_array(a_min,a_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,seq_length)).copy()
    
    for n in range(2,seq_length):
        vec_A = coeff_array[:,1:n]*seq_array[:,1:n]
        vec_B = seq_array[:,n-1:0:-1]
        seq_array[:,n] = (1/(n*(n-1)))*np.sum(vec_A*vec_B, axis=1)
    
    return seq_array

In [53]:
deg_1_sqnc(-1,-1,21)

array([[ 0.00000000e+00,  1.00000000e+00, -5.00000000e-01,
         2.50000000e-01, -1.25000000e-01,  6.25000000e-02,
        -3.12500000e-02,  1.56250000e-02, -7.81250000e-03,
         3.90625000e-03, -1.95312500e-03,  9.76562500e-04,
        -4.88281250e-04,  2.44140625e-04, -1.22070312e-04,
         6.10351562e-05, -3.05175781e-05,  1.52587891e-05,
        -7.62939453e-06,  3.81469727e-06, -1.90734863e-06]])

## Quadratic Case

In [4]:
def deg_2_array(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_squared_matrix = np.multiply.outer(a_range, u_range*u_range) # a x u
    au_tensor = np.broadcast_to(au_squared_matrix,(seq_length,b_len,a_len,seq_length)) # k x b x a x k
    #k,b,a,k --> a,b,k,k
    
    bu_matrix = np.multiply.outer(b_range,u_range) # b_len x seq_length
    buv_matrix = np.multiply.outer(bu_matrix,v_range) # b_len x seq_len x seq_len
    buv_tensor = np.broadcast_to(buv_matrix,(a_len,b_len,seq_length,seq_length))
    
    corrected_au_tensor = np.swapaxes(au_tensor, 0,2)
    #corrected_buv_tensor = np.swapaxes(buv_tensor, 2,3)
    
    tensor = corrected_au_tensor + buv_tensor
    
    P_array = np.swapaxes(tensor, 2,3)
    
    return P_array


def deg_2_sqnc(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_2_array(a_min,a_max,b_min,b_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 2, axis2 = 3)[:,:,1:n]
        vec_A = diag*seq_array[:,:,1:n]
        vec_B = seq_array[:,:,n-1:0:-1]
        seq_array[:,:,n] = (1/((n*n)*(n-1)))*np.sum(vec_A*vec_B,axis=2)
    
    return seq_array

In [5]:
deg_2_sqnc(-24,-24,36,36,21)

array([[[ 0.,  1.,  3.,  4.,  7.,  6., 12.,  8., 15., 13., 18., 12.,
         28., 14., 24., 24., 31., 18., 39., 20., 42.]]])

## Cubic Case

In [7]:
def deg_3_array(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_squared_matrix = np.multiply.outer(a_range, u_range*u_range*u_range) # a x u^3
    au_tensor = np.broadcast_to(au_squared_matrix,(seq_length,b_len,a_len,seq_length)) # k x b x a x k
    #k,b,a,k --> a,b,k,k
    
    bu_matrix = np.multiply.outer(b_range,u_range*u_range) # b x u^2
    buv_matrix = np.multiply.outer(bu_matrix,v_range) # b x u^2 x v
    buv_tensor = np.broadcast_to(buv_matrix,(a_len,b_len,seq_length,seq_length)) # a x b x k x k
    
    corrected_au_tensor = np.swapaxes(au_tensor, 0,2)
    #corrected_buv_tensor = np.swapaxes(buv_tensor, 2,3)
    
    tensor = corrected_au_tensor + buv_tensor
    
    P_array = np.swapaxes(tensor, 2,3)
    
    return P_array


def deg_3_sqnc(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_3_array(a_min,a_max,b_min,b_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 2, axis2 = 3)[:,:,1:n]
        vec_A = diag*seq_array[:,:,1:n]
        vec_B = seq_array[:,:,n-1:0:-1]
        seq_array[:,:,n] = (1/((n**3)*(n-1)))*np.sum(vec_A*vec_B,axis=2)
    
    return seq_array

In [9]:
deg_3_sqnc(-24,-24,48,48,21)

array([[[ 0.,  1.,  3.,  4.,  7.,  6., 12.,  8., 15., 13., 18., 12.,
         28., 14., 24., 24., 31., 18., 39., 20., 42.]]])

## Catalan Numbers

In [10]:
def catalan(length):
    catlist = np.zeros(length)
    catlist[1] = 1
    for n in range(2,length):
        vec_A = catlist[1:n]
        vec_B = catlist[n-1:0:-1]
        catlist[n] = np.dot(vec_A,vec_B)

    return catlist

In [11]:
catalan(21)

array([0.00000000e+00, 1.00000000e+00, 1.00000000e+00, 2.00000000e+00,
       5.00000000e+00, 1.40000000e+01, 4.20000000e+01, 1.32000000e+02,
       4.29000000e+02, 1.43000000e+03, 4.86200000e+03, 1.67960000e+04,
       5.87860000e+04, 2.08012000e+05, 7.42900000e+05, 2.67444000e+06,
       9.69484500e+06, 3.53576700e+07, 1.29644790e+08, 4.77638700e+08,
       1.76726319e+09])

## Checking for Geometric Sequences

In [56]:
def array_div(A): # input is 3d array
    if len(A.shape) == 2:
        orig_sqnc_len = A.shape[1]
        
        shift_1 = A[:,2::]
        shift_2 = A[:,1:-1]
        
        div_array = np.divide(shift_1,shift_2, out=np.full_like(shift_1,np.nan), where=shift_2!=0)
        
        return div_array
    
    elif len(A.shape) == 3:
        orig_sqnc_len = A.shape[2]

        shift_1 = A[:,:,2::] # starts each sequence at their second index
        shift_2 = A[:,:,1:-1] # then at their first index

        div_array = np.divide(shift_1,shift_2, out=np.full_like(shift_1,np.nan), where=shift_2!=0)
        # puts a zero wherever it had to divide by zero

        return div_array
    
    elif len(A.shape) == 4:
        orig_sqnc_len = A.shape[3]
        
        shift_1 = A[:,:,:,2::]
        shift_2 = A[:,:,:,1:-1]
        
        div_array = np.divide(shift_1,shift_2, out=np.full_like(shift_1,np.nan), where=shift_2!=0)
        
        return div_array

In [73]:
def geom_sqnc_check(A,a_min=0,b_min=0,c_min=0): # need those extra inputs to get the a and b values correct in the tuples
    np.seterr(all="ignore")
   
    quot_array = array_div(A)
    
    if len(A.shape) == 2:
        bool_array = (np.abs(quot_array[:,:-1] - quot_array[:,1:]) < 1e-5).sum(axis=1) == (quot_array.shape[1]-1)
        
        a_coords = np.where(bool_array)[0]
        length = len(a_coords)
        coord_list = [(a_coords[n]+a_min,a_coords[n]) for n in range(length)]
        
        return coord_list
    
    elif len(A.shape) == 3:
        bool_array = (np.abs(quot_array[:,:,:-1] - quot_array[:,:,1:]) < 1e-5).sum(axis=2) == (quot_array.shape[2]-1)

        arrays = np.where(bool_array)
        a_coords = arrays[0]
        b_coords = arrays[1]
        length = len(a_coords)
        coord_list = [(a_coords[n]+a_min,b_coords[n]+b_min,a_coords[n],b_coords[n]) for n in range(length)]
        # I wanted to include the indices as well so it's easier to find in the array

        return coord_list
    
    elif len(A.shape) == 4:
        bool_array = (np.abs(quot_array[:,:,:,:-1] - quot_array[:,:,:,1:]) < 1e-5).sum(axis=3) == (quot_array.shape[3]-1)
        
        arrays = np.where(bool_array)
        a_coords = arrays[0]
        b_coords = arrays[1]
        c_coords = arrays[2]
        length = len(a_coords)
        coord_list = [(a_coords[n]+a_min,b_coords[n]+b_min,c_coords[n]+c_min) for n in range(length)]
        # Just letting the tuple be (a,b,c) or else it'll get really cluttered
        
        return coord_list

In [102]:
A = deg_2_sqnc(-10,10,-10,10,21)
geom_sqnc_check(A,-10,-10,-10)

[(-10, -10, 0, 0),
 (-9, -9, 1, 1),
 (-8, -8, 2, 2),
 (-7, -7, 3, 3),
 (-6, -6, 4, 4),
 (-5, -5, 5, 5),
 (-4, -4, 6, 6),
 (-3, -3, 7, 7),
 (-2, -2, 8, 8),
 (-1, -1, 9, 9),
 (1, 1, 11, 11),
 (2, 2, 12, 12),
 (3, 3, 13, 13),
 (4, 4, 14, 14),
 (5, 5, 15, 15),
 (6, 6, 16, 16),
 (7, 7, 17, 17),
 (8, 8, 18, 18),
 (9, 9, 19, 19),
 (10, 10, 20, 20)]

In [69]:
geom_sqnc_check(deg_1_sqnc(-10,10,21),-10)

[(-10, 0),
 (-9, 1),
 (-8, 2),
 (-7, 3),
 (-6, 4),
 (-5, 5),
 (-4, 6),
 (-3, 7),
 (-2, 8),
 (-1, 9),
 (1, 11),
 (2, 12),
 (3, 13),
 (4, 14),
 (5, 15),
 (6, 16),
 (7, 17),
 (8, 18),
 (9, 19),
 (10, 20)]

In [70]:
array_div(deg_1_sqnc(673,673,21)) # a = 1000, b = -327

array([[336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5,
        336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5,
        336.5]])

It looks like the linear case *only* produces geometric sequences!

In [71]:
geom_sqnc_check(deg_2_sqnc(-10,10,-10,10,21),-10,-10)

[(-10, -10, 0, 0),
 (-9, -9, 1, 1),
 (-8, -8, 2, 2),
 (-7, -7, 3, 3),
 (-6, -6, 4, 4),
 (-5, -5, 5, 5),
 (-4, -4, 6, 6),
 (-3, -3, 7, 7),
 (-2, -2, 8, 8),
 (-1, -1, 9, 9),
 (1, 1, 11, 11),
 (2, 2, 12, 12),
 (3, 3, 13, 13),
 (4, 4, 14, 14),
 (5, 5, 15, 15),
 (6, 6, 16, 16),
 (7, 7, 17, 17),
 (8, 8, 18, 18),
 (9, 9, 19, 19),
 (10, 10, 20, 20)]

In [60]:
array_div(deg_4_sqnc(-10,10,-10,10,-10,10,21)).shape

(21, 21, 21, 19)

In [18]:
array_div(deg_2_sqnc(-10,10,-10,10,21))[16,16]

array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3.])

In [19]:
array_div(deg_2_sqnc(-10,10,-10,10,21))[16,4]

array([ 0., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan])

Looks like the quadratic function produces a geometric sequence for each ordered pair $(a,b) = (a,a)$ and sequences of zeros for $(a,b) = (a,-a)$, $a \in \mathbb{Z}$.

In [20]:
geom_sqnc_check(deg_3_sqnc(-30,30,-50,50,21),-30,-50)

[(-16, -48, 14, 2),
 (-15, -45, 15, 5),
 (-14, -42, 16, 8),
 (-13, -39, 17, 11),
 (-12, -36, 18, 14),
 (-11, -33, 19, 17),
 (-10, -30, 20, 20),
 (-9, -27, 21, 23),
 (-8, -24, 22, 26),
 (-7, -21, 23, 29),
 (-6, -18, 24, 32),
 (-5, -15, 25, 35),
 (-4, -12, 26, 38),
 (-3, -9, 27, 41),
 (-2, -6, 28, 44),
 (-1, -3, 29, 47),
 (1, 3, 31, 53),
 (2, 6, 32, 56),
 (3, 9, 33, 59),
 (4, 12, 34, 62),
 (5, 15, 35, 65),
 (6, 18, 36, 68),
 (7, 21, 37, 71),
 (8, 24, 38, 74),
 (9, 27, 39, 77),
 (10, 30, 40, 80),
 (11, 33, 41, 83),
 (12, 36, 42, 86),
 (13, 39, 43, 89),
 (14, 42, 44, 92),
 (15, 45, 45, 95),
 (16, 48, 46, 98)]

In [21]:
deg_3_sqnc(-24,-24,48,48,21) # finding our original sigma_n in the cubics

array([[[ 0.,  1.,  3.,  4.,  7.,  6., 12.,  8., 15., 13., 18., 12.,
         28., 14., 24., 24., 31., 18., 39., 20., 42.]]])

## Some Ideas

After finding a function that finds the geometric sequences, try dividing everything by sequences of catalan numbers or sequences of sums of factors, then put it through the function. This way we can pinpoint the combo-sequences.

### Finding Combo Sequences with $\sigma_n$

In [22]:
divsigma = deg_2_sqnc(-40,40,-40,40,21)
sigma = deg_2_sqnc(-24,-24,36,36,21)
divsigma[:,:,1::] = divsigma[:,:,1::]/sigma[:,:,1::]

geom_sqnc_check(divsigma,-40,-40)

[(-26, 39, 14, 79),
 (-24, 36, 16, 76),
 (-22, 33, 18, 73),
 (-20, 30, 20, 70),
 (-18, 27, 22, 67),
 (-16, 24, 24, 64),
 (-14, 21, 26, 61),
 (-12, 18, 28, 58),
 (-10, 15, 30, 55),
 (-8, 12, 32, 52),
 (-6, 9, 34, 49),
 (-4, 6, 36, 46),
 (-2, 3, 38, 43),
 (2, -3, 42, 37),
 (4, -6, 44, 34),
 (6, -9, 46, 31),
 (8, -12, 48, 28),
 (10, -15, 50, 25),
 (12, -18, 52, 22),
 (14, -21, 54, 19),
 (16, -24, 56, 16),
 (18, -27, 58, 13),
 (20, -30, 60, 10),
 (22, -33, 62, 7),
 (24, -36, 64, 4),
 (26, -39, 66, 1)]

First look at possible combo-sequences:

In [23]:
array_div(divsigma)[14,79] # a = -26, b = 39

array([1.08333333, 1.08333333, 1.08333333, 1.08333333, 1.08333333,
       1.08333333, 1.08333333, 1.08333333, 1.08333333, 1.08333333,
       1.08333333, 1.08333333, 1.08333333, 1.08333333, 1.08333333,
       1.08333333, 1.08333333, 1.08333333, 1.08333333])

Looks like $\big\{\left(\frac{13}{12}\right)^n\big\}$ for $a = -26$ and $b = 39$. Now to look at the original sequence in that place:

In [24]:
deg_2_sqnc(-40,40,-40,40,21)[14,79]

array([  0.        ,   1.        ,   3.25      ,   4.69444444,
         8.89988426,   8.26417824,  17.90571952,  12.93190854,
        26.26793923,  24.66267628,  36.99401441,  26.7178993 ,
        67.53691212,  36.58249406,  67.93891755,  73.60049401,
       102.98958016,  64.78376816, 152.06190027,  84.47883348,
       192.18934618])

In [25]:
array_div(deg_2_sqnc(-40,40,-40,40,21))[52,22]

array([-1.5       , -0.66666667, -0.875     , -0.42857143, -1.        ,
       -0.33333333, -0.9375    , -0.43333333, -0.69230769, -0.33333333,
       -1.16666667, -0.25      , -0.85714286, -0.5       , -0.64583333,
       -0.29032258, -1.08333333, -0.25641026, -1.05      ])

In [26]:
array_div(divsigma)[52,22]

array([-0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5,
       -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5])

Looks pretty evident that $a = 12$, $b = -18$ $\Rightarrow$ $\{-0.5^n\}\times\sigma(n)$. Now to find some more:

In [27]:
array_div(divsigma)[22,67]

array([0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75,
       0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75])

$a = -18$, $b = 27$ $\Rightarrow$ $\{0.75^n\}\times\sigma(n)$.

In [28]:
array_div(divsigma)[34, 49]

array([0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25])

$a = -6, b = 9$ $\Rightarrow$ $\{0.25^n\}\times\sigma(n)$

In [29]:
array_div(divsigma)[16, 76] # this is just where sigma_n would be

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1.])

In [30]:
array_div(divsigma)[38,43] # 0.08333333... = 1/12

array([0.08333333, 0.08333333, 0.08333333, 0.08333333, 0.08333333,
       0.08333333, 0.08333333, 0.08333333, 0.08333333, 0.08333333,
       0.08333333, 0.08333333, 0.08333333, 0.08333333, 0.08333333,
       0.08333333, 0.08333333, 0.08333333, 0.08333333])

$a = -2, \ b = 3 \ \Rightarrow \ \big\{\left(\frac{1}{12}\right)^n\big\}\times\sigma(n)$. Didn't want to show up from the function because of floating point inconsistencies.

In [31]:
array_div(divsigma)[28,58]

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

$a = -12$, $b = 18$ $\Rightarrow$ $\{0.5^n\}\times\sigma(n)$

In [32]:
array_div(divsigma)[58,13]

array([-0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75,
       -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75,
       -0.75])

$a = 18$, $b = -27$ $\Rightarrow$ $\{-0.75^n\}\times\sigma(n)$.

In [33]:
array_div(divsigma)[36,46] # 0.166666... = 1/6

array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667, 0.16666667, 0.16666667, 0.16666667])

$a = -4, \ b = 6 \ \Rightarrow \big\{\left(\frac{1}{6}\right)^n\big\}\times\sigma(n)$

What I'm seeing is that, for $\tau(n) = \frac{1}{n^2(n-1)}\sum_{u+v=n}(au^2+buv)\tau(u)\tau(v)$, $(a,b)=(a,a)$ produces a geometric sequence $\{\rho^n\}$ for $\rho \in \mathbb{Q}$, $(a,b) = (-24,36)$ produces $\sigma(n)$, and $(a,b) = (-2k,3k)$ for $k \in \mathbb{Z}$ produces some sequence $\{\rho^n\}\times\sigma(n)$. The next step would be to connect $a$ and $\rho$ as well as $k$ and $\rho$.

# Next Stuff

Play with other denominators, like $\frac{1}{n^3}$ or $\frac{1}{n(n-1)(n-2)}$. Also extend search to higher degree polynomials - quartics, quintics, etc. Put together a function that checks for sequences of integers.

### Integer Sequence Checker

In [109]:
def int_sqnc_check(A,a_min=0,b_min=0,c_min=0):
    if len(A.shape) == 2:
        bool_array = (np.abs(A - np.around(A)) < 1e-5).sum(axis=1) >= (A.shape[1]-1)

        a_coords = np.where(bool_array)[0]
        length = len(a_coords)
        coord_list = [(a_coords[n]+a_min,a_coords[n]) for n in range(length)]

        return coord_list
    
    elif len(A.shape) == 3:
        bool_array = (np.abs(A - np.around(A)) < 1e-5).sum(axis=2) >= (A.shape[2]-1)

        arrays = np.where(bool_array)
        a_coords = arrays[0]
        b_coords = arrays[1]
        length = len(a_coords)
        coord_list = [(a_coords[n]+a_min,b_coords[n]+b_min,a_coords[n],b_coords[n]) for n in range(length)]

        return coord_list
    
    elif len(A.shape) == 4:
        bool_array = (np.abs(A - np.around(A)) < 1e-5).sum(axis=3) >= (A.shape[3]-1)

        arrays = np.where(bool_array)
        a_coords = arrays[0]
        b_coords = arrays[1]
        c_coords = arrays[2]
        length = len(a_coords)
        coord_list = [(a_coords[n]+a_min,b_coords[n]+b_min,c_coords[n]+c_min) for n in range(length)]

        return coord_list

In [110]:
A = deg_4_sqnc(-10,10,-10,10,-10,10,21)
int_sqnc_check(A,-10,-10,-10)

[(-10, 0, 10),
 (-10, 1, 9),
 (-10, 2, 8),
 (-10, 3, 7),
 (-10, 4, 6),
 (-10, 5, 5),
 (-10, 6, 4),
 (-10, 7, 3),
 (-10, 8, 2),
 (-10, 9, 1),
 (-10, 10, 0),
 (-9, -1, 10),
 (-9, 0, 9),
 (-9, 1, 8),
 (-9, 2, 7),
 (-9, 3, 6),
 (-9, 4, 5),
 (-9, 5, 4),
 (-9, 6, 3),
 (-9, 7, 2),
 (-9, 8, 1),
 (-9, 9, 0),
 (-9, 10, -1),
 (-8, -2, 10),
 (-8, -1, 9),
 (-8, 0, 8),
 (-8, 1, 7),
 (-8, 2, 6),
 (-8, 3, 5),
 (-8, 4, 4),
 (-8, 5, 3),
 (-8, 6, 2),
 (-8, 7, 1),
 (-8, 8, 0),
 (-8, 9, -1),
 (-8, 10, -2),
 (-7, -3, 10),
 (-7, -2, 9),
 (-7, -1, 8),
 (-7, 0, 7),
 (-7, 1, 6),
 (-7, 2, 5),
 (-7, 3, 4),
 (-7, 4, 3),
 (-7, 5, 2),
 (-7, 6, 1),
 (-7, 7, 0),
 (-7, 8, -1),
 (-7, 9, -2),
 (-7, 10, -3),
 (-6, -4, 10),
 (-6, -3, 9),
 (-6, -2, 8),
 (-6, -1, 7),
 (-6, 0, 6),
 (-6, 1, 5),
 (-6, 2, 4),
 (-6, 3, 3),
 (-6, 4, 2),
 (-6, 5, 1),
 (-6, 6, 0),
 (-6, 7, -1),
 (-6, 8, -2),
 (-6, 9, -3),
 (-6, 10, -4),
 (-5, -5, 10),
 (-5, -4, 9),
 (-5, -3, 8),
 (-5, -2, 7),
 (-5, -1, 6),
 (-5, 0, 5),
 (-5, 1, 4),
 (-5, 2, 3),
 (-5

In [115]:
A[12,18,16]

array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1.])

In [116]:
B = deg_3_sqnc(-50,50,-50,50,21)
int_sqnc_check(B,-50,-50)

[(-50, 50, 0, 100),
 (-49, 49, 1, 99),
 (-48, 48, 2, 98),
 (-47, 47, 3, 97),
 (-46, 46, 4, 96),
 (-45, 45, 5, 95),
 (-44, 44, 6, 94),
 (-43, 43, 7, 93),
 (-42, 42, 8, 92),
 (-41, 41, 9, 91),
 (-40, 40, 10, 90),
 (-39, 39, 11, 89),
 (-38, 38, 12, 88),
 (-37, 37, 13, 87),
 (-36, 36, 14, 86),
 (-35, 35, 15, 85),
 (-34, 34, 16, 84),
 (-33, 33, 17, 83),
 (-32, 32, 18, 82),
 (-31, 31, 19, 81),
 (-30, 30, 20, 80),
 (-29, 29, 21, 79),
 (-28, 28, 22, 78),
 (-27, 27, 23, 77),
 (-26, 26, 24, 76),
 (-25, 25, 25, 75),
 (-24, 24, 26, 74),
 (-24, 48, 26, 98),
 (-23, 23, 27, 73),
 (-22, 22, 28, 72),
 (-21, 21, 29, 71),
 (-20, 20, 30, 70),
 (-19, 19, 31, 69),
 (-18, 18, 32, 68),
 (-17, 17, 33, 67),
 (-16, -48, 34, 2),
 (-16, 16, 34, 66),
 (-15, 15, 35, 65),
 (-14, 14, 36, 64),
 (-13, 13, 37, 63),
 (-12, -36, 38, 14),
 (-12, 12, 38, 62),
 (-11, 11, 39, 61),
 (-10, -30, 40, 20),
 (-10, 10, 40, 60),
 (-9, 9, 41, 59),
 (-8, -24, 42, 26),
 (-8, 8, 42, 58),
 (-7, 7, 43, 57),
 (-6, -18, 44, 32),
 (-6, 6, 44, 

In [118]:
B[62,86]

array([0.00000000e+00, 1.00000000e+00, 6.00000000e+00, 3.60000000e+01,
       2.16000000e+02, 1.29600000e+03, 7.77600000e+03, 4.66560000e+04,
       2.79936000e+05, 1.67961600e+06, 1.00776960e+07, 6.04661760e+07,
       3.62797056e+08, 2.17678234e+09, 1.30606940e+10, 7.83641641e+10,
       4.70184985e+11, 2.82110991e+12, 1.69266594e+13, 1.01559957e+14,
       6.09359740e+14])

In [55]:
len(B.shape) == 3

True

### Quartic Case

Now we're letting $P(u,v) = au^4 + bu^3v + cu^2v^2$. This instantly complicates things, since I'll need to include a new axis in my matrix, but it should involve the same idea. In the end, the ordered triplet $(a,b,c) = (-24,24,48)$ should yield our $\sigma(n)$.

I'll also need to create new sequence checkers (or add features to the current ones) for each of these to account for the extra axis.

In [74]:
def deg_4_array(a_min,a_max,b_min,b_max,c_min,c_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    c_len = c_max+1 - c_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    c_range = np.arange(c_min, c_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_matrix = np.multiply.outer(a_range, u_range**4) # a x u^4
    au_tensor = np.broadcast_to(au_matrix,(seq_length,b_len,c_len,a_len,seq_length)) # k x b x c x a x u^4
    #v,b,c,a,u --> a,b,c,v,u --> a,b,c,u,v
    
    bu_matrix = np.multiply.outer(b_range,u_range**3) # b x u^3
    buv_matrix = np.multiply.outer(bu_matrix,v_range) # b x u^3 x v
    buv_tensor = np.broadcast_to(buv_matrix,(a_len,c_len,b_len,seq_length,seq_length)) # a x c x b x u^3 x v
    # a,c,b,u,v --> a,b,c,u,v
    
    cu_matrix = np.multiply.outer(c_range,u_range**2) # c x u^2
    cuv_matrix = np.multiply.outer(cu_matrix,v_range**2) # c x u^2 x v^2
    cuv_tensor = np.broadcast_to(cuv_matrix,(a_len,b_len,c_len,seq_length,seq_length)) # a x b x c x u^2 x v^2
    
    corrected_au_tensor = np.swapaxes(np.swapaxes(au_tensor, 3,4),0,4)
    corrected_buv_tensor = np.swapaxes(buv_tensor, 1,2)
    
    tensor = corrected_au_tensor + corrected_buv_tensor + cuv_tensor
    
    #P_array = np.swapaxes(tensor, 3,4)
    
    return tensor


def deg_4_sqnc(a_min,a_max,b_min,b_max,c_min,c_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    c_len = c_max+1 - c_min
    
    coeff_array = deg_4_array(a_min,a_max,b_min,b_max,c_min,c_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,c_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 3, axis2 = 4)[:,:,:,1:n]
        vec_A = diag*seq_array[:,:,:,1:n]
        vec_B = seq_array[:,:,:,n-1:0:-1]
        seq_array[:,:,:,n] = (1/((n**4)*(n-1)))*np.sum(vec_A*vec_B,axis=3)
    
    return seq_array

In [75]:
deg_4_sqnc(-24,-24,24,24,48,48,21)

array([[[[ 0.,  1.,  3.,  4.,  7.,  6., 12.,  8., 15., 13., 18., 12.,
          28., 14., 24., 24., 31., 18., 39., 20., 42.]]]])

### Quintic Case

Home is at $(a,b,c) = (-24,0,120)$.

In [91]:
def deg_5_array(a_min,a_max,b_min,b_max,c_min,c_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    c_len = c_max+1 - c_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    c_range = np.arange(c_min, c_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_matrix = np.multiply.outer(a_range, u_range**5) # a x u^5
    au_tensor = np.broadcast_to(au_matrix,(seq_length,b_len,c_len,a_len,seq_length)) # k x b x c x a x u^4
    #v,b,c,a,u --> a,b,c,v,u --> a,b,c,u,v
    
    bu_matrix = np.multiply.outer(b_range,u_range**4) # b x u^4
    buv_matrix = np.multiply.outer(bu_matrix,v_range) # b x u^4 x v
    buv_tensor = np.broadcast_to(buv_matrix,(a_len,c_len,b_len,seq_length,seq_length)) # a x c x b x u^3 x v
    # a,c,b,u,v --> a,b,c,u,v
    
    cu_matrix = np.multiply.outer(c_range,u_range**3) # c x u^3
    cuv_matrix = np.multiply.outer(cu_matrix,v_range**2) # c x u^3 x v^2
    cuv_tensor = np.broadcast_to(cuv_matrix,(a_len,b_len,c_len,seq_length,seq_length)) # a x b x c x u^3 x v^2
    
    corrected_au_tensor = np.swapaxes(np.swapaxes(au_tensor, 3,4),0,4)
    corrected_buv_tensor = np.swapaxes(buv_tensor, 1,2)
    
    tensor = corrected_au_tensor + corrected_buv_tensor + cuv_tensor
    
    #P_array = np.swapaxes(tensor, 3,4)
    
    return tensor


def deg_5_sqnc(a_min,a_max,b_min,b_max,c_min,c_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    c_len = c_max+1 - c_min
    
    coeff_array = deg_5_array(a_min,a_max,b_min,b_max,c_min,c_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,c_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 3, axis2 = 4)[:,:,:,1:n]
        vec_A = diag*seq_array[:,:,:,1:n]
        vec_B = seq_array[:,:,:,n-1:0:-1]
        seq_array[:,:,:,n] = (1/((n**5)*(n-1)))*np.sum(vec_A*vec_B,axis=3)
    
    return seq_array

In [92]:
deg_5_sqnc(-24,-24,0,0,120,120,21)

array([[[[ 0.,  1.,  3.,  4.,  7.,  6., 12.,  8., 15., 13., 18., 12.,
          28., 14., 24., 24., 31., 18., 39., 20., 42.]]]])

### Playing with $\frac{1}{n^2(n-1)}$

Below are some alterations to that term of the function. We use these to check for some possible interesting sequences of integers.

In [121]:
def deg_2_alter1(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_2_array(a_min,a_max,b_min,b_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 2, axis2 = 3)[:,:,1:n]
        vec_A = diag*seq_array[:,:,1:n]
        vec_B = seq_array[:,:,n-1:0:-1]
        seq_array[:,:,n] = (1/(n**3))*np.sum(vec_A*vec_B,axis=2)
    
    return seq_array

In [131]:
A = deg_2_alter1(-50,50,-50,50,21)
geom_sqnc_check(A,-50,-50)

[]

In [138]:
def deg_2_alter2(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_2_array(a_min,a_max,b_min,b_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 2, axis2 = 3)[:,:,1:n]
        vec_A = diag*seq_array[:,:,1:n]
        vec_B = seq_array[:,:,n-1:0:-1]
        seq_array[:,:,n] = (1/((2*n)*(n**2-1)))*np.sum(vec_A*vec_B,axis=2)

    return seq_array

In [145]:
B = deg_2_alter2(-50,50,-50,50,21)
int_sqnc_check(B,-50,-50)

[(-50, 50, 0, 100),
 (-49, 49, 1, 99),
 (-48, 48, 2, 98),
 (-47, 47, 3, 97),
 (-46, 46, 4, 96),
 (-45, 45, 5, 95),
 (-44, 44, 6, 94),
 (-43, 43, 7, 93),
 (-42, 42, 8, 92),
 (-41, 41, 9, 91),
 (-40, 40, 10, 90),
 (-39, 39, 11, 89),
 (-38, 38, 12, 88),
 (-37, 37, 13, 87),
 (-36, 36, 14, 86),
 (-35, 35, 15, 85),
 (-34, 34, 16, 84),
 (-33, 33, 17, 83),
 (-32, 32, 18, 82),
 (-31, 31, 19, 81),
 (-30, 30, 20, 80),
 (-29, 29, 21, 79),
 (-28, 28, 22, 78),
 (-27, 27, 23, 77),
 (-26, 26, 24, 76),
 (-25, 25, 25, 75),
 (-24, 24, 26, 74),
 (-23, 23, 27, 73),
 (-22, 22, 28, 72),
 (-21, 21, 29, 71),
 (-20, 20, 30, 70),
 (-19, 19, 31, 69),
 (-18, 18, 32, 68),
 (-17, 17, 33, 67),
 (-16, 16, 34, 66),
 (-15, 15, 35, 65),
 (-14, 14, 36, 64),
 (-13, 13, 37, 63),
 (-12, 12, 38, 62),
 (-11, 11, 39, 61),
 (-10, 10, 40, 60),
 (-9, 9, 41, 59),
 (-8, 8, 42, 58),
 (-7, 7, 43, 57),
 (-6, 6, 44, 56),
 (-5, 5, 45, 55),
 (-4, 4, 46, 54),
 (-3, 3, 47, 53),
 (-2, 2, 48, 52),
 (-1, 1, 49, 51),
 (0, -48, 50, 2),
 (0, -36,

In [153]:
array_div(B)[50,2]

array([-4., -4., -4., -4., -4., -4., -4., -4., -4., -4., -4., -4., -4.,
       -4., -4., -4., -4., -4., -4.])

In [231]:
def deg_2_alter3(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_2_array(a_min,a_max,b_min,b_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 2, axis2 = 3)[:,:,1:n]
        vec_A = diag*seq_array[:,:,1:n]
        vec_B = seq_array[:,:,n-1:0:-1]
        seq_array[:,:,n] = (1/((n**2)*(n-1)*(n+1)))*np.sum(vec_A*vec_B,axis=2)

    return seq_array

In [237]:
B = deg_2_alter3(-50,50,-50,50,21)
int_sqnc_check(B,-50,-50)

[(-50, 50, 0, 100),
 (-49, 49, 1, 99),
 (-48, -48, 2, 2),
 (-48, 48, 2, 98),
 (-47, 47, 3, 97),
 (-46, 46, 4, 96),
 (-45, 45, 5, 95),
 (-44, 44, 6, 94),
 (-43, 43, 7, 93),
 (-42, 42, 8, 92),
 (-41, 41, 9, 91),
 (-40, 40, 10, 90),
 (-39, 39, 11, 89),
 (-38, 38, 12, 88),
 (-37, 37, 13, 87),
 (-36, -36, 14, 14),
 (-36, 36, 14, 86),
 (-35, 35, 15, 85),
 (-34, 34, 16, 84),
 (-33, 33, 17, 83),
 (-32, 32, 18, 82),
 (-31, 31, 19, 81),
 (-30, 30, 20, 80),
 (-29, 29, 21, 79),
 (-28, 28, 22, 78),
 (-27, 27, 23, 77),
 (-26, 26, 24, 76),
 (-25, 25, 25, 75),
 (-24, -24, 26, 26),
 (-24, 24, 26, 74),
 (-23, 23, 27, 73),
 (-22, 22, 28, 72),
 (-21, 21, 29, 71),
 (-20, 20, 30, 70),
 (-19, 19, 31, 69),
 (-18, 18, 32, 68),
 (-17, 17, 33, 67),
 (-16, 16, 34, 66),
 (-15, 15, 35, 65),
 (-14, 14, 36, 64),
 (-13, 13, 37, 63),
 (-12, -12, 38, 38),
 (-12, 12, 38, 62),
 (-11, 11, 39, 61),
 (-10, 10, 40, 60),
 (-9, 9, 41, 59),
 (-8, 8, 42, 58),
 (-7, 7, 43, 57),
 (-6, 6, 44, 56),
 (-5, 5, 45, 55),
 (-4, 4, 46, 54),

In [247]:
B[2,2]

array([ 0.00000000e+00,  1.00000000e+00, -8.00000000e+00,  4.80000000e+01,
       -2.56000000e+02,  1.28000000e+03, -6.14400000e+03,  2.86720000e+04,
       -1.31072000e+05,  5.89824000e+05, -2.62144000e+06,  1.15343360e+07,
       -5.03316480e+07,  2.18103808e+08, -9.39524096e+08,  4.02653184e+09,
       -1.71798692e+10,  7.30144440e+10, -3.09237645e+11,  1.30567006e+12,
       -5.49755814e+12])

Alright! So, to start off, we have an interesting sequence where $a = -48$ and $b = -48$. We get:
$$
\{\tau(n)\} = \{1,-8,48,-256,1280,6144,...\}
$$

In [249]:
B[74,74]

array([0.000000e+00, 1.000000e+00, 4.000000e+00, 1.200000e+01,
       3.200000e+01, 8.000000e+01, 1.920000e+02, 4.480000e+02,
       1.024000e+03, 2.304000e+03, 5.120000e+03, 1.126400e+04,
       2.457600e+04, 5.324800e+04, 1.146880e+05, 2.457600e+05,
       5.242880e+05, 1.114112e+06, 2.359296e+06, 4.980736e+06,
       1.048576e+07])

Another one for $(24,24)$:
$$
\{\tau(n)\} = \{1,4,12,32,80,192,448,...\}
$$
which should be a familiar sequence:
$$
\{\tau(n)\} = \{n2^{n-1}\}
$$

In [250]:
B[62,62]

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20.])

This just gives us the natural numbers. Maybe we have $\{\tau(n)\} = \{n1^{n-1}\}$.

In [251]:
B[86,86]

array([0.00000000e+00, 1.00000000e+00, 6.00000000e+00, 2.70000000e+01,
       1.08000000e+02, 4.05000000e+02, 1.45800000e+03, 5.10300000e+03,
       1.74960000e+04, 5.90490000e+04, 1.96830000e+05, 6.49539000e+05,
       2.12576400e+06, 6.90873300e+06, 2.23205220e+07, 7.17445350e+07,
       2.29582512e+08, 7.31794257e+08, 2.32452293e+09, 7.36098929e+09,
       2.32452293e+10])

$(36,36)$ gives $\{\tau(n)\} = \{n3^{n-1}\}.$

__Idea__: Let $\tau(n) = \frac{1}{n^2(n-1)(n+1)}\sum_{u+v=n}(au^2 +buv)\tau(u)\tau(v)$ be the $n^{th}$ term in a sequence. If $(a,b) = (12k,12k)$ for $k \in \mathbb{Z}$, then $\{\tau(n)\} = \{nk^{n-1}\}.$

In [422]:
def deg_2_alter4(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_2_array(a_min,a_max,b_min,b_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 2, axis2 = 3)[:,:,1:n]
        vec_A = diag*seq_array[:,:,1:n]
        vec_B = seq_array[:,:,n-1:0:-1]
        seq_array[:,:,n] = (12/((n-1)*n**2))*np.sum(vec_A*vec_B,axis=2)

    return seq_array

In [423]:
D = deg_2_alter4(-50,50,-50,50,21)
int_sqnc_check(D,-50,-50)

[(-50, -50, 0, 0),
 (-50, -45, 0, 5),
 (-50, -40, 0, 10),
 (-50, -35, 0, 15),
 (-50, -30, 0, 20),
 (-50, -25, 0, 25),
 (-50, -20, 0, 30),
 (-50, -15, 0, 35),
 (-50, -10, 0, 40),
 (-50, -5, 0, 45),
 (-50, 0, 0, 50),
 (-50, 5, 0, 55),
 (-50, 50, 0, 100),
 (-49, -49, 1, 1),
 (-49, -48, 1, 2),
 (-49, -39, 1, 11),
 (-49, -24, 1, 26),
 (-49, -14, 1, 36),
 (-49, 49, 1, 99),
 (-48, -48, 2, 2),
 (-48, -28, 2, 22),
 (-48, -27, 2, 23),
 (-48, -23, 2, 27),
 (-48, -3, 2, 47),
 (-48, 2, 2, 52),
 (-48, 4, 2, 54),
 (-48, 48, 2, 98),
 (-47, -47, 3, 3),
 (-47, -42, 3, 8),
 (-47, -28, 3, 22),
 (-47, -22, 3, 28),
 (-47, -17, 3, 33),
 (-47, -3, 3, 47),
 (-47, 47, 3, 97),
 (-46, -46, 4, 4),
 (-46, -31, 4, 19),
 (-46, -29, 4, 21),
 (-46, -21, 4, 29),
 (-46, -17, 4, 33),
 (-46, 4, 4, 54),
 (-46, 46, 4, 96),
 (-45, -50, 5, 0),
 (-45, -45, 5, 5),
 (-45, -40, 5, 10),
 (-45, -35, 5, 15),
 (-45, -30, 5, 20),
 (-45, -25, 5, 25),
 (-45, -20, 5, 30),
 (-45, -15, 5, 35),
 (-45, -10, 5, 40),
 (-45, -5, 5, 45),
 (-45, 0

In [451]:
D[74,14]-np.around(D[74,14]) # (24,-36)

array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -3.7252903e-09,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00])

16,41

In [484]:
A = deg_2_alter4(2,2,47,47,140)
A - np.around(A)

B = A/deg_2_sqnc(-24,-24,36,36,140)
B

array([[[            nan, 1.00000000e+000, 4.90000000e+001,
         4.85100000e+003, 3.60549000e+005, 5.43536910e+007,
         3.50063816e+009, 6.75177257e+011, 4.62511260e+013,
         6.84958020e+015, 6.34617755e+017, 1.22074474e+020,
         6.70738559e+021, 1.71949001e+024, 1.28547446e+026,
         1.64723374e+028, 1.63399940e+030, 3.60538438e+032,
         2.13176616e+034, 5.32511222e+036, 3.24819770e+038,
         5.46080025e+040, 6.21730825e+042, 1.19447897e+045,
         6.11943499e+046, 1.51692165e+049, 1.43393184e+051,
         1.92824627e+053, 1.76389122e+055, 4.21666717e+057,
         2.25000207e+059, 6.48313318e+061, 4.21705299e+063,
         7.08791793e+065, 8.06812254e+067, 1.16232533e+070,
         7.85103500e+071, 2.40758165e+074, 1.95257120e+076,
         2.67892790e+078, 2.13449468e+080, 5.85699182e+082,
         3.28123206e+084, 9.16722455e+086, 6.14882105e+088,
         8.47921171e+090, 1.17623740e+093, 2.25923912e+095,
         1.11984098e+097, 3.11943920e+09

In [485]:
B-np.around(B)

array([[[       nan,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ,  0.       ,  0.       , -0.1171875,  0.       ,
          0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
          0.       ,  0.       ,  

In [487]:
A-np.around(A)

array([[[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.  ,
         0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
         0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
         0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
         0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
         0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
         0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
         0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
         0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
         0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
         0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
         0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
         0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
         0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]]])

$$
\tau(n) = \frac{12}{n^2(n-1)}\sum_{u+v=n}(2u^2+47uv)\tau(u)\tau(v) = \frac{1}{n^2(n-1)}\sum_{u+v=n}(24u^2 + 546uv)\tau(u)\tau(v)
$$
As is (hopefully) shown above, $\tau(n) = \varphi(n)\sigma(n)$, where $\varphi(n)$ is some kind of sequence of integers??? I don't know about that 9th term, but it seems to stabilize as integers after that.

In [488]:
D[54,94] - np.around(D[54,94])

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.375,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   ])

$$
\tau(n) = \frac{12}{n^2(n-1)}\sum_{u+v=n}(4u^2+44uv)\tau(u)\tau(v) = \frac{1}{n^2(n-1)}\sum{u+v=n}(24u^2 + 528uv)\tau(u)\tau(v)
$$
Same thing happens here, even with the 9th term. I have no idea what these are all about, or whether these are even sequences of integers at all.